In [35]:
# Patch the starter to improve Windows support and fix the download button
from pathlib import Path
import textwrap, io, json


base = Path(r"C:\Users\ryassminh\python_notebooks\Dashboard")

# --- Update README with Windows-specific instructions ---
readme_path = base.joinpath("README.md")
readme = readme_path.read_text(encoding="utf-8")

readme_path.write_text(textwrap.dedent("""
# Streamlit Feature Engineering Starter

This template helps you **prototype features first** and then reuse them in a Streamlit dashboard.

## How to run 

### 1) Create & activate a virtual environment

**Windows PowerShell**
```powershell
python -m venv .venv
.\\.venv\\Scripts\\Activate.ps1
``` 
**Windows Command Prompt**
#If activation is blocked, run once as admin:
```cmd
Set-ExecutionPolicy -ExecutionPolicy RemoteSigned -Scope CurrentUser
```

 """)
)


452

In [3]:
#!python -m venv .venv
!.\\.venv\\Scripts\\activate.bat

In [23]:
%%writefile etl/etl_cbmc1.py
from __future__ import annotations

import argparse
import logging
from dataclasses import dataclass
from datetime import date
from pathlib import Path
from typing import Iterable, List, Optional

import pandas as pd

try:
    import duckdb  # optional
except Exception:
    duckdb = None

# -------------------------
# Config & constants
# -------------------------

REQUIRED_COLUMNS = [
    "C1_BANNER_ID",
    "C1_ACADEMIC_PERIOD_DESC",
    "C1_ACADEMIC_PERIOD",
    "C1_CALENDAR_YEAR",
    "C1_COLLEGE",
    "C1_GENDER_DESC",
    "C1_CURRENT_AGE",
    "C1_FTIC_DC_DESC",
    "C1_TYPE_MAJOR_DESC",
    "C1_FTPT_COLLEGE_CENSUS",
    "C1_THECB_ETHNICITY",
]

REQUIRED_COLUMNS_STU=[
    "Term", 
    "Student ID",
    "Major Desc",
                      
]

COLUMN_MAPPING = {
    "C1_CALENDAR_YEAR": "Calendar Year",
    "C1_ACADEMIC_PERIOD_DESC": "Academic Period",
    "C1_CBM_TERM_DESC": "Term",
    "C1_COLLEGE": "SPC College",
    "C1_GENDER_DESC": "Gender",
    "C1_FTIC_DC_DESC": "Student Type",
    "C1_TYPE_MAJOR_DESC": "Major Type",
    "C1_FTPT_COLLEGE_CENSUS": "Full_Part Time",
    "C1_THECB_ETHNICITY": "Ethnicity",
    "C1_CURRENT_AGE": "Age",
    "Major Desc":"Major",
}

AGE_BINS = [0, 18, 25, 30, 35, 40, 50, 60, 200]
AGE_LABELS = ["Under 18", "18-24", "25-29", "30-34", "35-39", "40-49", "50-59", "60+"]


@dataclass
class ETLPaths:
    raw_inputs: List[Path]            # explicit files OR directories to glob under
    stu_inputs: List[Path]            # explicit files OR directories to glob under
    out_parquet: Path                 # curated parquet output
    out_csv: Path                     # curated csv output
    duckdb_path: Optional[Path] = None  # optional duckdb file
    duckdb_table: str = "cbmc1_merged"


# -------------------------
# Logging
# -------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)-8s | %(message)s",
)
logger = logging.getLogger("cbmc1_etl")


# -------------------------
# Utilities
# -------------------------

def _iter_input_files(inputs: Iterable[Path]) -> Iterable[Path]:
    """Yield CSV/XLS/XLSX files from paths (files or directories)."""
    for p in inputs:
        p = Path(p)
        if p.is_dir():
            # read all CSV + Excel in this directory (one-level glob)
            for f in p.glob("*.csv"):
                yield f
            for f in p.glob("*.xls*"):
                yield f
        elif p.is_file():
            if p.suffix.lower() in {".csv", ".xls", ".xlsx"}:
                yield p
        else:
            logger.warning("Path does not exist: %s", p)


def _read_one_file(fp: Path, columns: List[str]) -> pd.DataFrame:
    """Read a single CSV/Excel, subset columns, robust to encodings and date parsing."""
    suffix = fp.suffix.lower()
    try:
        if suffix == ".csv":
            # fast-path: try utf-8, fall back to latin1
            try:
                df = pd.read_csv(fp, dtype=str, low_memory=False)
            except UnicodeDecodeError:
                df = pd.read_csv(fp, dtype=str, encoding="latin1", low_memory=False)
        elif suffix in {".xlsx", ".xls"}:
            # engine auto-chooses; for xls you may need xlrd installed
            df = pd.read_excel(fp, dtype=str)
        else:
            raise ValueError(f"Unsupported file type: {fp}")

        missing = [c for c in columns if c not in df.columns]
        if missing:
            logger.warning("Missing columns in %s: %s", fp.name, missing)

        keep = [c for c in columns if c in df.columns]
        df = df[keep].copy()
        df["__source_file"] = fp.name  # lineage
        return df

    except Exception as e:
        logger.exception("Failed to read %s: %s", fp, e)
        # Return empty DF with required columns so concat is safe
        return pd.DataFrame(columns=columns + ["__source_file"])


def extract_merge(inputs: Iterable[Path], columns: List[str] = REQUIRED_COLUMNS) -> pd.DataFrame:
    """Extract: read many CSV/Excel → vertical concat, subsetting columns early."""
    files = list(_iter_input_files(inputs))
    if not files:
        logger.error("No input files found.")
        return pd.DataFrame(columns=columns)

    logger.info("Found %d files. Reading...", len(files))
    dfs = [_read_one_file(fp, columns) for fp in files]
    merged = pd.concat(dfs, ignore_index=True)
    logger.info("Merged shape after extract: %s", merged.shape)
    return merged


def _age_from_dob(dob_series: pd.Series) -> pd.Series:
    """
    Compute exact age in years (month/day aware) from a single column (Series)
    of DATE_OF_BIRTH values.
    Works even if some dates are missing or invalid.
    """
    # 1. Convert column to datetime (handles strings, Excel serials, NaNs)
    dob = pd.to_datetime(dob_series, errors="coerce")

    # 2. Get today's date (scalar timestamp)
    today = pd.Timestamp.today()

    # 3. Extract components (safe for Series)
    year = dob.dt.year
    month = dob.dt.month
    day = dob.dt.day

    # 4. Compute base difference in years
    diff = today.year - year

    # 5. Subtract 1 if birthday hasn't occurred yet this year
    had_birthday = (today.month > month) | ((today.month == month) & (today.day >= day))
    age = diff - (~had_birthday).astype("Int64")

    # 6. Keep null where dob is NaT
    return age.where(dob.notna()).astype("Int64")



def merge_student_data(
    cbm_df: pd.DataFrame,
    stu_df: pd.DataFrame,
    left_keys: list[str] = ['C1_ACADEMIC_PERIOD', 'C1_BANNER_ID'],
    right_keys: list[str] = ['Term', 'Student ID'],
    how: str = 'left',
    suffixes: tuple[str, str] = ("_cbm", "_stu"),
) -> pd.DataFrame:
    """
    Merge CBM dataset with student dataset safely and cleanly.

    Parameters
    ----------
    cbm_df : pd.DataFrame
        The main CBM dataset.
    stu_df : pd.DataFrame
        The student data to join (lookup/enrichment).
    left_keys : list[str]
        Column names in cbm_df to join on.
    right_keys : list[str]
        Column names in stu_df to join on.
    how : str
        Type of join (default 'left').
    suffixes : tuple[str, str]
        Suffixes to apply to overlapping columns.
    
    Returns
    -------
    pd.DataFrame
        Merged DataFrame with duplicates removed based on left_keys.
    """
    try:
        logger.info(f"Merging on {left_keys} ↔ {right_keys} using '{how}' join...")

        merged = pd.merge(
            cbm_df,
            stu_df,
            left_on=left_keys,
            right_on=right_keys,
            how=how,
            suffixes=suffixes
        )

        before = len(merged)
        merged = merged.drop_duplicates(subset=left_keys, keep='first')
        after = len(merged)

        logger.info(f"Merge complete. Rows: {before} → {after} after deduplication.")
        return merged.reset_index(drop=True)

    except KeyError as e:
        logger.error(f"KeyError during merge: {e}")
        missing_keys = [k for k in left_keys if k not in cbm_df.columns] + \
                       [k for k in right_keys if k not in stu_df.columns]
        raise KeyError(f"Missing join keys: {missing_keys}") from e

    except Exception as e:
        logger.exception("Unexpected error during merge:")
        raise



def transform_cbmc1(df: pd.DataFrame) -> pd.DataFrame:
    """
    Transform: the merged dataframe
      - rename columns
      - compute Age + Age_Group (and drop raw DOB & Age)
      - trim whitespace, standardize categories
      - drop all-null rows
    """
    
    if df.empty:
        logger.warning("Empty dataframe passed to transform; returning as-is.")
        return df

    # Standardize column names & trim strings
    df = df.drop_duplicates().copy()
    # basic whitespace cleanup
    for col in df.columns:
        if df[col].dtype == object:
            df[col] = df[col].astype(str).str.strip()

    # rename only existing columns
    existing_map = {k: v for k, v in COLUMN_MAPPING.items() if k in df.columns}
    df = df.rename(columns=existing_map)

    # age + age group
    if "Age" in df.columns:
        age = pd.to_numeric(df["Age"], errors="coerce").astype("Int64")
        df["Age_Group"] = pd.cut(
            age.astype("float"),
            bins=AGE_BINS,
            labels=AGE_LABELS,
            include_lowest=True,
            right=False,
        )
        df = df.drop(columns=["Age"], errors="ignore")
        # If you want to keep exact age, uncomment:
        # df["Age"] = age

    # Drop rows completely empty
    df = df.dropna().drop_duplicates().reset_index(drop=True)

    # Optional: cast some columns to category for smaller Parquet
    for cat_col in ["Term", "SPC College", "Gender", "Student Type", "Type_Major", "Full_Part_Time", "ETHNICITY", "Age_Group","Major"]:
        if cat_col in df.columns:
            df[cat_col] = df[cat_col].astype("category")

    logger.info("Transformed shape: %s | Columns: %s", df.shape, list(df.columns))
    return df


def load_parquet(df: pd.DataFrame, out_path_par: Path,out_path_csv: Path) -> Path:
    out_path_par.parent.mkdir(parents=True, exist_ok=True)
    # filter out feature not need to show 
    col_delete=['__source_file_cbm','C1_BANNER_ID','__source_file_stu','Student ID','SPC College','C1_ACADEMIC_PERIOD']
    df = df.drop(columns=col_delete, errors="ignore").dropna().reset_index(drop=True)
    # Use pyarrow by default if available
    
    df.to_parquet(out_path_par, index=False)
    df.to_csv(out_path_csv, index=False)
    logger.info("Wrote curated Parquet → %s (rows=%s)", out_path_par, len(df))
    return "save files in data/curated"


def load_duckdb(df: pd.DataFrame, db_path: Path, table: str) -> Optional[Path]:
    if duckdb is None:
        logger.warning("duckdb not installed; skipping DuckDB load.")
        return None
    db_path.parent.mkdir(parents=True, exist_ok=True)
    con = duckdb.connect(str(db_path))
    try:
        con.register("df_mem", df)
        con.execute(f"CREATE OR REPLACE TABLE {table} AS SELECT * FROM df_mem;")
        logger.info("Wrote DuckDB table %s in %s", table, db_path)
    finally:
        con.close()
    return db_path


def run_etl(paths: ETLPaths) -> Path:
    raw_df = extract_merge(paths.raw_inputs, REQUIRED_COLUMNS) # Extract + merge
    stu_df = extract_merge(paths.stu_inputs, REQUIRED_COLUMNS_STU) #
    input_df = merge_student_data(
    cbm_df=raw_df,
    stu_df=stu_df,
    left_keys=['C1_ACADEMIC_PERIOD', 'C1_BANNER_ID'],
    right_keys=['Term', 'Student ID'],
    how='left') # Merge CBM + Student data
    curated = transform_cbmc1(input_df) #
    out = load_parquet(curated, paths.out_parquet,paths.out_csv.with_suffix('.csv'))# Load Parquet (and CSV)
    if paths.duckdb_path:
        load_duckdb(curated, paths.duckdb_path, paths.duckdb_table)
    return out


# -------------------------
# CLI
# -------------------------

def _parse_args() -> argparse.Namespace:
    p = argparse.ArgumentParser(description="CBMC1 ETL: merge CSV/Excel → Parquet (and optional DuckDB).")
    p.add_argument(
        "--cbmc_inputs",
        nargs="+",
        required=True,
        help="Files or directories (space-separated). Directories will be scanned for *.csv and *.xls*", 
        
    )
    p.add_argument(
        "--stu_inputs",
        nargs="+",
        required=True,
        help="Files or directories (space-separated). Directories will be scanned for *.csv and *.xls*", 
    
    )
    p.add_argument(
        "--out_parquet",
        required=True,
        help="Path to curated parquet, e.g. data/curated/merged.parquet",
    )    
    p.add_argument(
        "--out_csv",
        required=True,
        help="Path to curated csv, e.g. data/curated/merged.csv",    
    )
    p.add_argument(
        "--duckdb",
        default=None,
        help="Optional DuckDB file path, e.g. warehouse/warehouse.duckdb",
    )
    p.add_argument(
        "--duckdb-table",
        default="cbmc1_merged",
        help="DuckDB table name (default: cbmc1_merged)",
    )
    return p.parse_args()


def main():
    args = _parse_args()
    paths = ETLPaths(
        raw_inputs=[Path(p) for p in args.cbmc_inputs],
        stu_inputs=[Path(p) for p in args.stu_inputs],
        out_parquet=Path(args.out_parquet),
        out_csv=Path(args.out_csv),
        duckdb_path=Path(args.duckdb) if args.duckdb else None,
        duckdb_table=args.duckdb_table,
    )
    run_etl(paths)


if __name__ == "__main__":
    main()


Overwriting etl/etl_cbmc1.py


In [ ]:
!python etl/etl_cbmc1.py \
  --cbmc_inputs  data/cbmc1/"cbmc1_Fall23_25.csv" \
  --stu_inputs  data/stu220/"STU0220_Fall24_25.csv" \
  --out_parquet data/curated/data_merged.parquet \
  --out_csv data/curated/data_merged.csv \
  --duckdb warehouse/warehouse.duckdb \
  --duckdb-table cbmc1_merged


2025-10-08 10:14:06,096 | INFO     | Found 1 files. Reading...
2025-10-08 10:14:08,794 | INFO     | Merged shape after extract: (28609, 12)
2025-10-08 10:14:08,796 | INFO     | Found 1 files. Reading...
2025-10-08 10:14:09,986 | INFO     | Merged shape after extract: (53758, 4)
2025-10-08 10:14:09,986 | INFO     | Merging on ['C1_ACADEMIC_PERIOD', 'C1_BANNER_ID'] ↔ ['Term', 'Student ID'] using 'left' join...
2025-10-08 10:14:10,019 | INFO     | Merge complete. Rows: 28609 → 14535 after deduplication.
2025-10-08 10:14:10,116 | INFO     | Transformed shape: (14535, 16) | Columns: ['C1_BANNER_ID', 'Academic Period', 'C1_ACADEMIC_PERIOD', 'Calendar Year', 'SPC College', 'Gender', 'Student Type', 'Major Type', 'Full_Part Time', 'Ethnicity', '__source_file_cbm', 'Term', 'Student ID', 'Major', '__source_file_stu', 'Age_Group']
2025-10-08 10:14:10,231 | INFO     | Wrote curated Parquet → data\curated\data_merged.parquet (rows=14535)
2025-10-08 10:14:10,295 | INFO     | Wrote DuckDB table cbmc1

# Dashboard application code

In [ ]:
%%writefile App.py
#https://student-insights.streamlit.app/
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from datetime import datetime
#from etl.etl_cbmc1 import  ETLPaths, run_etl

##############################################
COLUMNS=[
"Academic Period",
"Calendar Year",
"Gender",
"Student Type",
"Full_Part Time",
"Ethnicity",
"Major",
"Age_Group",
    
]


def create_bar_chart(data, x_col, y_col, title="Bar Chart", color_col=None, 
                     orientation='vertical', color_scheme='viridis'):
    """
    Create a customizable bar chart for Streamlit dashboard
    
    Parameters:
    - data: DataFrame with the data
    - x_col: Column name for x-axis
    - y_col: Column name for y-axis (values)
    - title: Chart title
    - color_col: Column name for color grouping (optional)
    - orientation: 'vertical' or 'horizontal'
    - color_scheme: Color palette ('viridis', 'blues', 'reds', etc.)
    """
    
    if orientation == 'horizontal':
        fig = px.bar(data, x=y_col, y=x_col, 
                     color=color_col if color_col else None,
                     orientation='h',
                     title=title,
                     color_discrete_sequence=px.colors.qualitative.Set3)
    else:
        fig = px.bar(data, x=x_col, y=y_col, 
                     color=color_col if color_col else None,
                     title=title,
                     color_discrete_sequence=px.colors.qualitative.Set3)
    
    # Customize the layout
    fig.update_layout(
        title_font_size=20,
        title_x=0.5,  # Center the title
        xaxis_title_font_size=14,
        yaxis_title_font_size=14,
        font_size=12,
        plot_bgcolor='rgba(0,0,0,0)',  # Transparent background
        paper_bgcolor='rgba(0,0,0,0)',
        height=500
    )
    
    # Add hover information
    fig.update_traces(
        hovertemplate='<b>%{x}</b><br>Value: %{y}<extra></extra>'
    )
    
    return fig
###
def create_stacked_bar_chart(data, x_col, y_cols, title="Stacked Bar Chart"):
    """Create a stacked bar chart with multiple y-columns"""
    fig = go.Figure()
    
    for col in y_cols:
        fig.add_trace(go.Bar(
            name=col,
            x=data[x_col],
            y=data[col]
        ))
    
    fig.update_layout(
        title=title,
        barmode='stack',
        height=500
    )
    
    return fig

def create_grouped_bar_chart(data, x_col, y_cols, title="Grouped Bar Chart"):
    """Create a grouped bar chart with multiple y-columns"""
    fig = go.Figure()
    
    for col in y_cols:
        fig.add_trace(go.Bar(
            name=col,
            x=data[x_col],
            y=data[col]
        ))
    
    fig.update_layout(
        title=title,
        barmode='group',
        height=500
    )


##

# =============================================================================
# CACHED FUNCTIONS FOR PERFORMANCE OPTIMIZATION
# =============================================================================

# Cache the data loading function to avoid reloading data every time the app runs
@st.cache_data
def load_and_process_data(file_par,columns=COLUMNS):
    """
    Load and process CBM data with caching for performance
    
    The @st.cache_data decorator ensures this function only runs once per session.
    Subsequent calls will return the cached result, dramatically improving performance.
    
    Returns:
        pd.DataFrame: Processed CBM data or None if loading fails
    """
    try:

        sample_data=(pd.read_parquet(file_par)[columns])

        
        
        return sample_data
    except Exception as e:
        # Display error message to user if data loading fails
        st.error(f"Error loading data: {str(e)}")
        return None

# Cache the value counts calculation to avoid recalculating for the same column
@st.cache_data
def get_value_counts(data, column):
    """
    Get value counts for a specific column with caching
    
    This function calculates how many times each unique value appears in a column.
    Caching prevents recalculation when the user switches between different view options
    (like percentages vs counts) for the same column.
    
    Args:
        data (pd.DataFrame): The dataset to analyze
        column (str): Name of the column to count values for
    
    Returns:
        pd.Series: Value counts for the specified column
    """
    return data[column].value_counts()

# Cache cross-tabulation calculations to improve performance for category comparisons
@st.cache_data
def get_cross_tabulation(data, col1, col2):
    """
    Get cross-tabulation between two columns with caching
    
    Cross-tabulation shows the relationship between two categorical variables
    by counting occurrences of each combination. Caching this expensive operation
    prevents recalculation when users switch between visualization options.
    
    Args:
        data (pd.DataFrame): The dataset to analyze
        col1 (str): First categorical column
        col2 (str): Second categorical column
    
    Returns:
        pd.DataFrame: Cross-tabulation table showing relationships between categories
    """
    ct = pd.crosstab(data[col1], data[col2])
    ct = ct[ct.sum(axis=0).sort_values(ascending=False).index]
    
    return ct


def sidebar_major_selector(df: pd.DataFrame,
                           major_col: str = "Major",
                           title: str = "🎓 Majors",
                           key_prefix: str = "majors") -> list[str]:
    """Sidebar widget: 'Select All' + multiselect for majors.

    - De-dupes & sorts options
    - Preserves selection in session state
    - Disables the multiselect when 'Select All' is checked
    - Guards against empty selections
    """
    # Build stable, clean options
    majors = (
        df[major_col]
        .dropna()
        .astype(str)
        .str.strip()
        .loc[lambda s: s.ne("")]
        .unique()
        .tolist()
    )
    majors = sorted(set(majors))  # de-dup + sort once

    with st.sidebar:
        st.subheader(title)
        # Select-all checkbox
        all_key = f"{key_prefix}_select_all"
        multi_key = f"{key_prefix}_multiselect"

        select_all = st.checkbox("Select all majors", value=True, key=all_key)

        if select_all:
            # Show disabled multiselect for clarity (everything selected)
            st.multiselect(
                "Majors",
                options=majors,
                default=majors,
                key=multi_key,
                disabled=True,
                help="All majors are included."
            )
            selected = majors
        else:
            # Use prior selection if exists; otherwise default to all
            default_vals = st.session_state.get(multi_key, majors)
            selected = st.multiselect(
                "Majors",
                options=majors,
                default=default_vals,
                key=multi_key,
                help="Uncheck 'Select all majors' to filter."
            )
            # Guard: if user clears everything, keep empty but warn
            if len(selected) == 0:
                st.info("No majors selected — results may be empty.")

    return selected




# =============================================================================
# MAIN DASHBOARD FUNCTION
# =============================================================================

def main():
    """
    Main function that creates the Streamlit dashboard for SPC data analysis.
    
    This function sets up the entire user interface, loads data, creates visualizations,
    and handles user interactions. It's optimized for performance with large datasets
    through caching, sampling, and smart data limiting.
    """
    
    # Configure the Streamlit page with title and wide layout for better space utilization
    st.set_page_config(page_title="Student Insights", layout="wide")
    col1, col2 = st.columns([1, 4])  
    with col1:
        # Display the SPC logo from local assets
        st.image("assets/analysis.png", width=200)
        
    with col2:     
        # Create the main title and separator line
        st.title("Student Insights", anchor=None)
        st.markdown(
            """
            <div style='color:#555;font-size:20px; line-height:1.4;margin-top:-10px'>
              <strong>An interactive dashboard designed to explore student enrollment, demographics, and program trends. </strong>
              <br>
              <strong> Use the sidebar filters to select academic terms, majors, and student characteristics to uncover patterns that support data-informed decisions. </strong>
            </div>
            """,
            unsafe_allow_html=True,
        )
    st.markdown("---")  # Creates a horizontal line for visual separation
    
    # =============================================================================
    # DATA LOADING SECTION
    # =============================================================================
    
    # Load data with caching - this spinner shows while data is loading
    # The spinner improves user experience by indicating that something is happening
    with st.spinner("Loading data..."):
        sample_data = load_and_process_data("data/curated/data_merged.parquet")
    
    # Check if data loading was successful
    if sample_data is None or sample_data.empty:
        # Display error message and stop execution if no data available
        st.error("Failed to load data. Please check the file path and try again.")
        return
    
    # =============================================================================
    # SESSION STATE OPTIMIZATION
    # =============================================================================
    
    # Store frequently used data information in session state to avoid repeated calculations
    # Session state persists across user interactions, improving performance
    if 'data_info' not in st.session_state:
        st.session_state.data_info = {
            'total_records': len(sample_data),  # Total number of rows in dataset
            'columns': sample_data.columns.tolist(),  # All column names
            # Only categorical columns (object/category types) are suitable for counting analysis
            'categorical_columns': sample_data.select_dtypes(include=['object', 'category']).columns.tolist()
        }
    
    # Use cached categorical columns list for better performance
    categorical_columns = st.session_state.data_info['categorical_columns']
    
    # =============================================================================
    # SIDEBAR CONTROLS SECTION
    # =============================================================================
    
    # Create sidebar for user controls - keeps main area clean for visualizations
    st.sidebar.header("Analysis Controls")
    
    # Dropdown to select which categorical column to analyze
    # Index=0 means the first column is selected by default
    selected_column = st.sidebar.selectbox(
        "Select Category to Analyze",
        categorical_columns,
        index=0
    )
    
    # Radio buttons for chart orientation - affects how bars are displayed
    orientation = st.sidebar.radio(
        "Chart Orientation",
        ["vertical", "horizontal"]  # vertical = bars go up, horizontal = bars go sideways
    )
    
    #checkbox to taggle between Terms
    show_terms = st.sidebar.multiselect(
        "Select Terms to Include",
        options=sample_data["Academic Period"].unique(),
        default=sample_data["Academic Period"].unique().tolist()  # Select all terms by default
    )
    #checkbox to taggle between Majors
    show_majors = sidebar_major_selector(sample_data, major_col="Major")
    
    # Checkbox to toggle between showing counts vs percentages
    show_percentages = st.sidebar.checkbox("Show Percentages", value=False)
    
    #----------------------------
    #fiter based on select terms 
    #--------------------------------
    if show_terms:
        sample_data = sample_data[sample_data["Academic Period"].isin(show_terms) & sample_data['Major'].isin(show_majors)]
    else:
        st.warning("Please select at least one term to display data.")
        return
    #-----------------------
    
    # =============================================================================
    # PERFORMANCE OPTIMIZATION FOR LARGE DATASETS
    # =============================================================================
    
    # If dataset is very large (>10,000 records), offer sampling option for better performance
    if len(sample_data) > 10000:
        # Inform user about large dataset and optimization options
        st.sidebar.info(f"Dataset has {len(sample_data):,} records. Using optimized processing.")
        
        # Checkbox to enable/disable sampling
        use_sampling = st.sidebar.checkbox("Use sampling for faster processing", value=True)
        
        if use_sampling:
            # Use maximum of 5,000 records or the full dataset size, whichever is smaller
            sample_size = min(5000, len(sample_data))
            # random_state=42 ensures reproducible sampling
            display_data = sample_data.sample(n=sample_size, random_state=42)
            st.sidebar.info(f"Using sample of {sample_size:,} records")
        else:
            # Use full dataset if sampling is disabled
            display_data = sample_data
    else:
        # For smaller datasets, use all data
        display_data = sample_data
        
        
    
    # =============================================================================
    # MAIN DASHBOARD LAYOUT
    # =============================================================================
    
    # Create two columns: left for chart (2/3 width), right for statistics (1/3 width)
    col1, col2 = st.columns([2, 1])
    
    # =============================================================================
    # LEFT COLUMN: MAIN VISUALIZATION
    # =============================================================================
    
    with col1:
        # Get cached value counts for the selected column
        # This prevents recalculation when user changes visualization options
        value_counts = get_value_counts(display_data, selected_column)
        
        # Limit categories for performance - showing too many bars makes charts unreadable
        if len(value_counts) > 20:
            st.info(f"Showing top 20 categories out of {len(value_counts)} total")
            value_counts = value_counts.head(20)  # Keep only top 20 most frequent categories
        
        # Convert value counts to DataFrame format required by Plotly
        count_data = value_counts.reset_index()
        count_data.columns = [selected_column, 'Count']  # Rename columns for clarity
        
        # Calculate percentages if user requested them
        if show_percentages:
            # Calculate percentage of each category relative to total
            count_data['Percentage'] = (count_data['Count'] / count_data['Count'].sum()) * 100
            y_col = 'Percentage'  # Use percentage column for y-axis
            title = f"Distribution of {selected_column} (%)"
            # Custom hover template showing both count and percentage
            hover_template = '<b>%{x}</b><br>Count: %{customdata}<br>Percentage: %{y:.1f}%<extra></extra>'
            customdata = count_data['Count']  # Show actual counts in hover
        else:
            # Use raw counts
            y_col = 'Count'
            title = f"Count of Each {selected_column}"
            # Simpler hover template for count-only display
            hover_template = '<b>%{x}</b><br>Count: %{y}<extra></extra>'
            customdata = None
        
        # Create bar chart using Plotly Express (faster than Graph Objects)
        if orientation == 'horizontal':
            # Horizontal bar chart: x-axis = values, y-axis = categories
            fig = px.bar(
                count_data, 
                x=y_col,  # Values (count or percentage)
                y=selected_column,  # Categories
                orientation='h',  # 'h' = horizontal bars
                title=title,
                color=y_col,  # Color bars by their height (creates gradient effect)
                color_continuous_scale='viridis'  # Professional color scheme
            )
        else:
            # Vertical bar chart: x-axis = categories, y-axis = values
            fig = px.bar(
                count_data, 
                x=selected_column,  # Categories
                y=y_col,  # Values (count or percentage)
                title=title,
                color=y_col,  # Color bars by their height
                color_continuous_scale='viridis'
            )
        
        # Customize chart appearance for better performance and aesthetics
        fig.update_layout(
            title_font_size=16,  # Readable title size
            height=450,  # Fixed height for consistent layout
            showlegend=False,  # Remove color legend to save space and improve performance
            plot_bgcolor='rgba(0,0,0,0)',  # Transparent plot background
            paper_bgcolor='rgba(0,0,0,0)'  # Transparent paper background
        )
        
        # Add custom hover information
        if customdata is not None:
            # When showing percentages, include both count and percentage in hover
            fig.update_traces(customdata=customdata, hovertemplate=hover_template)
        else:
            # When showing counts only, use simpler hover template
            fig.update_traces(hovertemplate=hover_template)
        
        # Add text annotations on bars only for small datasets to avoid clutter
        if len(count_data) <= 15:
            if show_percentages:
                # Show both count and percentage on bars
                text_values = [f"{count}<br>({pct:.1f}%)" for count, pct in 
                              zip(count_data['Count'], count_data['Percentage'])]
            else:
                # Show only counts on bars
                text_values = count_data['Count'].astype(str)
            
            # Position text outside bars for vertical, inside for horizontal orientation
            fig.update_traces(
                text=text_values,
                textposition='outside' if orientation == 'vertical' else 'inside'
            )
        
        # Display the chart using full container width
        st.plotly_chart(fig, use_container_width=True)
    
    # =============================================================================
    # RIGHT COLUMN: SUMMARY STATISTICS AND DATA TABLE
    # =============================================================================
    
    with col2:
        st.subheader("Summary Statistics")
        
        # Get summary statistics using cached data info for better performance
        #total_count = st.session_state.data_info['total_records']  # Total records in original dataset

        total_count = len(display_data) # Total records in original dataset
        unique_categories = display_data[selected_column].nunique()  # Number of unique values
        
        # Handle case where value_counts might be empty (error prevention)
        if not value_counts.empty:
            most_common = value_counts.index[0]  # Most frequent category
            most_common_count = value_counts.iloc[0]  # Count of most frequent category
        else:
            most_common = "N/A"
            most_common_count = 0
        
        # Display key metrics using Streamlit's metric widget for nice formatting
        st.metric("Total Records", f"{total_count:,}")  # :, adds thousand separators
        st.metric("Unique Categories", f"{unique_categories}")
        st.metric("Most Common", str(most_common))  # Convert to string for display
        st.metric("Most Common Count", f"{most_common_count:,}")
        
        # Show frequency table with the data
        st.subheader("Frequency Table")
        freq_table = count_data.copy()  # Copy to avoid modifying original data
        
        # Round percentages if they exist
        if 'Percentage' in freq_table.columns:
            freq_table['Percentage'] = freq_table['Percentage'].round(1)
        
        # Limit table size for performance - tables with many rows are slow to render
        if len(freq_table) > 10:
            freq_table_display = freq_table.head(10)  # Show only top 10 rows
            st.info(f"Showing top 10 out of {len(freq_table)} categories")
        else:
            freq_table_display = freq_table
            
        def auto_height(df, row_height=35, max_height=600):
            return min(len(df) * row_height + 40, max_height)
        # Display the frequency table with fixed height for consistent layout
        st.dataframe(freq_table_display, use_container_width=True, height=auto_height(freq_table_display))
    
    # =============================================================================
    # CROSS-CATEGORY ANALYSIS SECTION
    # =============================================================================
    
    # Add visual separator and section header
    st.markdown("---")
    st.subheader("Cross-Category Analysis")
    
    # Disable cross-analysis for very large datasets to maintain performance
    if len(display_data) > 50000:
        st.warning("Cross-category analysis disabled for large datasets to maintain performance.")
        return  # Exit function early
    
    # Create two columns for category selection
    col_a, col_b = st.columns(2)
    
    with col_a:
        # Dropdown for first category in comparison
        category_1 = st.selectbox("First Category", categorical_columns, index=0)
    
    with col_b:
        # Dropdown for second category, defaulting to second column if available
        if len(categorical_columns) > 1:
            default_index = 1 if len(categorical_columns) > 1 else 0
            category_2 = st.selectbox("Second Category", categorical_columns, index=default_index)
        else:
            # Show warning if not enough categorical columns for cross-analysis
            st.warning("Need at least 2 categorical columns for cross-analysis")
            return  # Exit function early
    
    # Only proceed if user selected two different categories
    if category_1 != category_2:
        # Check if cross-tabulation would be too large (performance safeguard)
        unique_cat1 = display_data[category_1].nunique()  # Number of unique values in first category
        unique_cat2 = display_data[category_2].nunique()  # Number of unique values in second category
        
        # Limit cross-tabulation size to prevent performance issues
        if unique_cat1 * unique_cat2 > 500:  # 500 cells is reasonable limit
            st.warning(f"Cross-tabulation too large ({unique_cat1} x {unique_cat2}). "
                      "Please select categories with fewer unique values.")
            return  # Exit function early
        
        # Create two columns for cross-analysis visualization and data
        col3, col4 = st.columns(2)
        
        # =============================================================================
        # LEFT COLUMN: CROSS-TABULATION VISUALIZATION
        # =============================================================================
        
        with col3:
            # Get cached cross-tabulation to avoid recalculation
            cross_tab = get_cross_tabulation(display_data, category_1, category_2)
            
            # Limit categories shown in visualization for readability and performance
            if len(cross_tab.index) > 10:
                cross_tab = cross_tab.head(10)  # Keep only top 10 rows
                st.info("Showing top 10 categories for performance")
            
            if len(cross_tab.columns) > 10:
                cross_tab = cross_tab.iloc[:, :10]  # Keep only first 10 columns
                st.info("Showing top 10 subcategories for performance")
            
            # Create stacked bar chart using Plotly Express for better performance
            # Transpose (.T) the data for better visualization
            fig2 = px.bar(
                cross_tab.T,  # Transpose so categories become x-axis
                title=f"{category_1} vs {category_2} Distribution",
                height=400  # Fixed height for consistency
            )
            
            # Customize chart layout
            fig2.update_layout(
                xaxis_title=category_2,  # X-axis shows second category
                yaxis_title="Count",     # Y-axis shows counts
                showlegend=True,         # Show legend for first category
                legend_title=category_1  # Legend title is first category
            )
            
            
            # Display the cross-tabulation chart
            st.plotly_chart(fig2, use_container_width=True)
        
        # =============================================================================
        # RIGHT COLUMN: CROSS-TABULATION DATA TABLE
        # =============================================================================
        
        with col4:
            st.subheader("Cross-Tabulation")
            
            # Show preview for large tables to maintain performance
            if cross_tab.size > 100:  # If table has more than 100 cells
                st.info("Showing preview of cross-tabulation")
                # Show only 5x5 preview of the full table
                st.dataframe(cross_tab.iloc[:5, :5], use_container_width=True)
            else:
                # Show full table for smaller cross-tabulations
                st.dataframe(cross_tab, use_container_width=True)
            
            # Show summary statistics instead of full proportions table (better performance)
            st.subheader("Summary")
            st.write(f"**Categories in {category_1}:** {len(cross_tab.index)}")
            st.write(f"**Categories in {category_2}:** {len(cross_tab.columns)}")
            st.write(f"**Total Combinations:** {cross_tab.sum().sum():,}")  # Total count across all cells

# =============================================================================
# SCRIPT ENTRY POINT
# =============================================================================

if __name__ == "__main__":
    # Run the main function when script is executed directly
    main()

Overwriting App.py


In [10]:
!streamlit run App.py

^C
